In [25]:
import arcpy
from dataclasses import dataclass

# An object that represents a list of features found at a given station
# __dict__ to return a dictionary of all the fields
# fields to return user defined list of fields
# __list__ to return a list of field values in order

FEET = arcpy.LinearUnitConversionFactor("Feet", "Meters")

class Feature:
    """Class for single feature objects"""
    path: str
    fields: list[str]
    _default_values: list[str]

    def __init__(self, path=None, fields=[], default_values={}):

        # Object attributes
        self.path = path
        self.fields = fields
        self._default_values = default_values

        # List all fields in the feature class
        self._all_fields = [field.name.replace('Shape', 'SHAPE@') 
                            for field in arcpy.ListFields(self.path)]
        
        # Test for missing fields
        if set(fields) <= set(self._all_fields):
            self.row = {field: self._default_values.get(field, None)
                        for field in self.fields}
        else:
            # Raise error showing which field(s) are missing
            badFields = [field 
                         for field in fields 
                         if field not in self._all_fields]
            raise ValueError(f"Fields: {badFields} were not found in {self.path}!")
     
    def cursorRow(self):
        return list(self.row.values())
    
    # Get value from field
    def __getitem__(self, field):
        return self.row[field]
    
    # Set value for field
    def __setitem__(self, field, new_value):
        self.row[field] = new_value
    
class StationPoint(Feature):

    _default_values = {
        "ProFeature": ", , "
    }

    # Initialize first with default values, then with specified values
    def __init__(self, path, fields, stationing):
        super().__init__(path, fields, self._default_values)
        self['Stationing'] = stationing

    # 
    def __repr__(self):
        return f"<Station point at station: {self['Stationing']} with features {self['ProFeature']} and shape: {self['SHAPE@']}>"

centerPoint = None
path = r"C:\\Users\\danda\\OneDrive\\Documents\\ArcGIS\\Projects\\COH\\Profile_Builder.gdb\\Profile_StationPoints"

STA = StationPoint(path, ['Stationing', 'ProFeature', 'testField', 'SHAPE@'], 370)

STA['SHAPE@'] = 'TEST'

print(STA.fields)
print(STA.cursorRow())
print(STA)
for field in STA.fields:
    print(f"{field}: {STA[field]}")


['Stationing', 'ProFeature', 'testField', 'SHAPE@']
[370, ', , ', None, 'TEST']
<Station point at station: 370 with features , ,  and shape: TEST>
Stationing: 370
ProFeature: , , 
testField: None
SHAPE@: TEST
